<h1>SJ Training fNIRS Data Analysis</h1>
Written by Ansley Kunnath

Run with 3.9.19 conda

In [130]:
# Install packages if you need to; otherwise skip this block

#import subprocess
#import sys

#def install(packages):
#    for package in packages:
#        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

#packages = ['mne', 'PyQt5', 'numpy', 'matplotlib', 'scipy', 'pooch', 'pyarrow', 
#            'pandas', 'statsmodels', 'mne_nirs', 'tables', 'pyvista', 'pyvistaqt', 
#            'darkdetect', 'xlrd']

# Don't need to install itertools, collections, os, time, warnings, glob, csv
# Can also do !pip install mne

#print("done")

done


In [1]:
# Import packages

import PyQt5
import numpy as np
import matplotlib.pyplot as plt
from itertools import compress
from collections import defaultdict
from mne.viz import plot_compare_evokeds
import os
from os import path as op
import time
import warnings
from scipy import signal, stats
import pooch
import pyarrow
import pandas as pd
with warnings.catch_warnings(record=True):
    warnings.simplefilter('ignore', FutureWarning)
    from nilearn.glm.first_level import \
        make_first_level_design_matrix, compute_regressor  # noqa
import statsmodels.formula.api as smf
import mne_nirs.preprocessing
import mne_nirs.statistics
import mne_nirs.utils
import mne_nirs.statistics
import mne
from mne.preprocessing.nirs import tddr
import glob
import csv
import warnings
import tables
#warnings.filterwarnings("ignore", category=DeprecationWarning) 
#warnings.filterwarnings("ignore", category=FutureWarning) 

%matplotlib qt

print("done")

done


In [2]:
# Locate files

subjects = ('201 202 203 204 205 206 207 208 209 212 213 214 215 216 217 218 219 221 223 224').split() #
# Mapping of subjects to groups
subject_to_group = {
    201: "trained",
    202: "control",
    203: "trained",
    204: "control",
    205: "control",
    206: "control",
    207: "trained",
    208: "control",
    209: "control",
    212: "trained",
    213: "trained",
    214: "trained",
    215: "control",
    216: "trained",
    217: "control",
    218: "control",
    219: "trained",
#    220: "control", #sampling rate = 2.40 Hz instead of 4.8
    221: "trained",
    223: "trained",
    224: "control",
}

sfreq = 4.807692
conditions = ('A', 'V', 'AV', 'W')
groups = ('trained','control')
days = ('1', '3')
runs = (1, 2)

condition_colors = dict(  # https://personal.sron.nl/~pault/data/colourschemes.pdf
    A='#4477AA',  # sblue
    AV='#CCBB44',  # yellow
    V='#EE7733',  # orange
    W='#AA3377',  # purple
)
exp_name = 'av'
duration = 1.8
design = 'event'
#plot_subject = '207'
#plot_run = 1
beh_title, beh_idx = 'AV', 0
filt_kwargs = dict(
    l_freq=0.02, l_trans_bandwidth=0.02,
    h_freq=0.2, h_trans_bandwidth=0.02)
run_h = True  # regenerate HbO/HbR
n_jobs = 4  # for GLM

# SET FOLDER LOCATIONS
# I save the output files outside of the folder that's uploaded to Github
current_directory = %pwd
os.chdir(current_directory)

raw_path = 'data'
proc_path = 'processed'
results_path = 'results'
subjects_dir = 'subjects'
os.makedirs(proc_path, exist_ok=True)
os.makedirs(results_path, exist_ok=True)
os.makedirs(subjects_dir, exist_ok=True)
mne.datasets.fetch_fsaverage(subjects_dir=subjects_dir, verbose=True)

use = None
all_sci = list()
plt.rcParams['axes.titlesize'] = 8
plt.rcParams['axes.labelsize'] = 8
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
got_bad = 0
got_total = 0

# Prep making bad channels report
bad_channels_filename = op.join(results_path, 'bad_channels_report.csv')
with open(bad_channels_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Subject', 'Day', 'Run', 'Percent Bad'])
def normalize_channel_names(channels_set):
    return {name.split()[0] for name in channels_set}

# Sanity check for subjects
subjects_check = {int(subject) for subject in subjects}
subject_to_group_check = set(subject_to_group.keys())
if subjects_check == subject_to_group_check:
    print("done")
    del subjects_check
    del subject_to_group_check
else:
    print("Error loading subject info") 

0 files missing from root.txt in /Users/wallacelab/Documents/Ansley/fnirs-analysis/subjects
0 files missing from bem.txt in /Users/wallacelab/Documents/Ansley/fnirs-analysis/subjects/fsaverage
done


In [3]:
###############################################################################
# Load participant data

for subject in subjects:
    for day in days:
        for run in runs:
            group = subject_to_group.get(int(subject), "unknown")
            root1 = f'Day{day}'
            root2 = f'{subject}_{day}'
            root3 = f'*-*-*_{run:03d}'
            fname_base = op.join(raw_path, root1, root2, root3)
            fname = glob.glob(fname_base)
            base = f'{subject}_{day}_{run:03d}'
            base_pr = base.ljust(20)
            raw_intensity = mne.io.read_raw_nirx(fname[0])
            raw_od = mne.preprocessing.nirs.optical_density(
                raw_intensity, verbose='error')
            # identify bad channels
            peaks = np.ptp(raw_od.get_data('fnirs'), axis=-1)
            flat_names = [
                raw_od.ch_names[f].split(' ')[0]
                for f in np.where(peaks < 0.001)[0]]
            sci = mne.preprocessing.nirs.scalp_coupling_index(raw_od)
            all_sci.extend(sci)
            sci_mask = (sci < 0.25)
            got = np.where(sci_mask)[0]
            percentage_bad = (len(got) / len(raw_od.ch_names)) * 100
            print(f'    Run {base_pr}')
            # assign bads
            assert raw_od.info['bads'] == []
            bads = set(raw_od.ch_names[pick] for pick in got)
            bads = bads | set(ch_name for ch_name in raw_od.ch_names
                            if ch_name.split(' ')[0] in flat_names)
            bads = sorted(bads)
            # Convert OD to [Hb]
            raw_h = mne.preprocessing.nirs.beer_lambert_law(raw_od, 6.) # test - BLL
            # Run TDDR and band-pass filter
            raw_tddr = tddr(raw_h) # test - tddr
            raw_tddr_bp = raw_tddr.copy().filter(**filt_kwargs) # test -- BPF
#            raw_tddr = tddr(raw_od) # DON'T TDDR?
#            raw_tddr_bp = raw_tddr.copy()
#            raw_tddr_bp = raw_tddr.copy().filter(**filt_kwargs) # DON'T BANDPASS FILTER?
#            raw_tddr_bp.info['bads'] = bads
            picks = mne.pick_types(raw_tddr_bp.info, fnirs=True)
            peaks = np.ptp(raw_tddr_bp.get_data(picks), axis=-1)
            assert (peaks > 1e-9).all()
            raw_tddr_bp.info['bads'] = [] 
#            raw_h = mne.preprocessing.nirs.beer_lambert_law(raw_tddr_bp, 6.) 
            h_bads = [
                ch_name for ch_name in raw_tddr_bp.ch_names
                if ch_name.split(' ')[0] in set(bad.split(' ')[0] for bad in bads)]
            set_bads = set(bads) 
            set_h_bads = set(h_bads)
            normalized_bads = normalize_channel_names(set_bads)
            normalized_h_bads = normalize_channel_names(set_h_bads)
            assert normalized_bads == normalized_h_bads
            raw_tddr_bp.info['bads'] = h_bads
            raw_tddr_bp.info._check_consistency()
            picks = mne.pick_types(raw_tddr_bp.info, fnirs=True)
            peaks = np.ptp(raw_tddr_bp.get_data(picks), axis=-1)
            assert (peaks > 1e-9).all()  
            raw_h_interp = raw_tddr_bp.copy().interpolate_bads(reset_bads=False, method = dict(fnirs = 'nearest')) # TEST
                # True = bads are removed
                # False = interpolated channels ARE included in analyses
            raw_h_interp.save(op.join(proc_path, f'{base}_hbo_raw.fif'), 
                       overwrite=True)  
            assert len(raw_tddr_bp.ch_names) == len(raw_h_interp.ch_names)
            # Save the bad channel data
            with open(bad_channels_filename, mode='a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([subject, day, run, f'{percentage_bad:.2f}%'])
            del raw_intensity, raw_od, raw_tddr_bp, raw_h, raw_tddr, raw_h_interp
            del normalized_h_bads, normalized_bads, set_bads, set_h_bads


Loading data/Day1/201_1/2023-09-21_001
    Run 201_1_001           
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.02 - 0.2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.02
- Lower transition bandwidth: 0.02 Hz (-6 dB cutoff frequency: 0.01 Hz)
- Upper passband edge: 0.20 Hz
- Upper transition bandwidth: 0.02 Hz (-6 dB cutoff frequency: 0.21 Hz)
- Filter length: 795 samples (165.360 s)

Setting channel interpolation method to {'fnirs': 'nearest'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 93.6 mm
Writing /Users/wallacelab/Documents/Ansley/fnirs-analysis/processed/201_1_001_hbo_raw.fif
Closing /Users/wallacelab/Documents/Ansley/fnirs-analysis/processed/201_1_001_hbo_raw.fif
[done]
Loading data/Day1/201_1/2023-09-21_002


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.0s


    Run 201_1_002           
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.02 - 0.2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.02
- Lower transition bandwidth: 0.02 Hz (-6 dB cutoff frequency: 0.01 Hz)
- Upper passband edge: 0.20 Hz
- Upper transition bandwidth: 0.02 Hz (-6 dB cutoff frequency: 0.21 Hz)
- Filter length: 795 samples (165.360 s)

Setting channel interpolation method to {'fnirs': 'nearest'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 93.6 mm
Writing /Users/wallacelab/Documents/Ansley/fnirs-analysis/processed/201_1_002_hbo_raw.fif
Closing /Users/wallacelab/Documents/Ansley/fnirs-analysis/processed/201_1_002_hbo_raw.fif
[done]
Loading data/Day3/201_3/2023-09-23_001


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.0s


    Run 201_3_001           
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.02 - 0.2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.02
- Lower transition bandwidth: 0.02 Hz (-6 dB cutoff frequency: 0.01 Hz)
- Upper passband edge: 0.20 Hz
- Upper transition bandwidth: 0.02 Hz (-6 dB cutoff frequency: 0.21 Hz)
- Filter length: 795 samples (165.360 s)

Setting channel interpolation method to {'fnirs': 'nearest'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 93.6 mm
Writing /Users/wallacelab/Documents/Ansley/fnirs-analysis/processed/201_3_001_hbo_raw.fif
Closing /Users/wallacelab/Documents/Ansley/fnirs-analysis/processed/201_3_001_hbo_raw.fif
[done]
Loading data/Day3/201_3/2023-09-23_002


...

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.0s
/var/folders/4k/pq2dd_x1047dwxhfm5_xl8rw0000gn/T/ipykernel_38591/644534609.py:65: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  raw_h_interp = raw_tddr_bp.copy().interpolate_bads(reset_bads=False, method = dict(fnirs = 'nearest')) # TEST


    Run 224_3_001           
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.02 - 0.2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.02
- Lower transition bandwidth: 0.02 Hz (-6 dB cutoff frequency: 0.01 Hz)
- Upper passband edge: 0.20 Hz
- Upper transition bandwidth: 0.02 Hz (-6 dB cutoff frequency: 0.21 Hz)
- Filter length: 795 samples (165.360 s)

Setting channel interpolation method to {'fnirs': 'nearest'}.
Writing /Users/wallacelab/Documents/Ansley/fnirs-analysis/processed/224_3_001_hbo_raw.fif
Closing /Users/wallacelab/Documents/Ansley/fnirs-analysis/processed/224_3_001_hbo_raw.fif
[done]
Loading data/Day3/224_3/2024-02-12_002


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.1s
/var/folders/4k/pq2dd_x1047dwxhfm5_xl8rw0000gn/T/ipykernel_38591/644534609.py:65: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  raw_h_interp = raw_tddr_bp.copy().interpolate_bads(reset_bads=False, method = dict(fnirs = 'nearest')) # TEST


    Run 224_3_002           
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.02 - 0.2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.02
- Lower transition bandwidth: 0.02 Hz (-6 dB cutoff frequency: 0.01 Hz)
- Upper passband edge: 0.20 Hz
- Upper transition bandwidth: 0.02 Hz (-6 dB cutoff frequency: 0.21 Hz)
- Filter length: 795 samples (165.360 s)

Setting channel interpolation method to {'fnirs': 'nearest'}.
Writing /Users/wallacelab/Documents/Ansley/fnirs-analysis/processed/224_3_002_hbo_raw.fif
Closing /Users/wallacelab/Documents/Ansley/fnirs-analysis/processed/224_3_002_hbo_raw.fif
[done]


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.0s
/var/folders/4k/pq2dd_x1047dwxhfm5_xl8rw0000gn/T/ipykernel_38591/644534609.py:65: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  raw_h_interp = raw_tddr_bp.copy().interpolate_bads(reset_bads=False, method = dict(fnirs = 'nearest')) # TEST


In [4]:
# Remove patients with over 25% bad channels on average across days and runs

subjects_to_remove = ['202', '203', '204', '206', '214', '221', '223']

# Initialize counters for each group
removed_trained = 0
removed_control = 0

# Count and remove the subjects
for subject in subjects_to_remove:
    subject_int = int(subject)  # Convert to integer for dictionary key comparison
    if subject_int in subject_to_group:
        # Increment the appropriate counter based on the group of the subject
        if subject_to_group[subject_int] == "trained":
            removed_trained += 1
        elif subject_to_group[subject_int] == "control":
            removed_control += 1
        # Remove the subject from the dictionary
        subject_to_group.pop(subject_int, None)

# Update the subjects list after counting the removed subjects
subjects = [subject for subject in subjects if subject not in subjects_to_remove]

# Output the results
print(f'Removed {removed_trained} trained subjects.')
print(f'Removed {removed_control} control subjects.')

Removed 4 trained subjects.
Removed 3 control subjects.


In [5]:
###############################################################################
# Load events

for subject in subjects:
    for day in days:
        for run in runs:
            fname = op.join(proc_path, f'{subject}_{day}_{run:03d}_hbo_raw.fif')
            raw_h = mne.io.read_raw_fif(fname)
            events, _ = mne.events_from_annotations(raw_h)
            #print(len(events))


Opening raw data file processed/201_1_001_hbo_raw.fif...
    Range : 0 ... 3994 =      0.000 ...   830.752 secs
Ready.
Used Annotations descriptions: ['1.0', '2.0', '3.0', '4.0', '5.0']
Opening raw data file processed/201_1_002_hbo_raw.fif...
    Range : 0 ... 3838 =      0.000 ...   798.304 secs
Ready.
Used Annotations descriptions: ['1.0', '2.0', '3.0', '4.0', '5.0']
Opening raw data file processed/201_3_001_hbo_raw.fif...
    Range : 0 ... 3945 =      0.000 ...   820.560 secs
Ready.
Used Annotations descriptions: ['1.0', '2.0', '3.0', '4.0', '5.0']
Opening raw data file processed/201_3_002_hbo_raw.fif...
    Range : 0 ... 3945 =      0.000 ...   820.560 secs
Ready.
Used Annotations descriptions: ['1.0', '2.0', '3.0', '4.0', '5.0']
Opening raw data file processed/205_1_001_hbo_raw.fif...
    Range : 0 ... 4171 =      0.000 ...   867.568 secs
Ready.
Used Annotations descriptions: ['1.0', '2.0', '3.0', '4.0', '5.0']
Opening raw data file processed/205_1_002_hbo_raw.fif...
    Range : 0

In [7]:
###############################################################################
# Channel example figure
sfreq = 4.807692

subject = '215'
run = '2'
day = '3'


def _make_design(raw_h, design, subject=None, run=None, day=None, group=None):
    annotations_to_remove = raw_h.annotations.description == '255.0'
    raw_h.annotations.delete(annotations_to_remove)
    events, _ = mne.events_from_annotations(raw_h)
    rows_to_remove = events[:, -1] == 1
    events = events[~rows_to_remove]
    # mis-codings
    if len(events)==101:
        events = events[1:]
    n_times = len(raw_h.times)
    stim = np.zeros((n_times, 4))
    events[:, 2] -= 1
    assert len(events) == 100, len(events)
    want = [0] + [25] * 4
    count = np.bincount(events[:, 2])
    assert np.array_equal(count, want), count
    assert events.shape == (100, 3), events.shape
    if design == 'block':
        events = events[0::5]
        duration = 20.
        assert np.array_equal(np.bincount(events[:, 2]), [0] + [5] * 4)
    else:
        assert design == 'event'
        assert len(events) == 100
        duration = 1.8
        assert events.shape == (100, 3)
        events_r = events[:, 2].reshape(20, 5)
        assert (events_r == events_r[:, :1]).all()
        del events_r
    idx = (events[:, [0, 2]] - [0, 1]).T
    assert np.in1d(idx[1], np.arange(len(conditions))).all()
    stim[tuple(idx)] = 1
    raw_h.info['sfreq'] == sfreq  # necessary for below logic to work
    n_block = int(np.ceil(duration * sfreq))
    stim = signal.fftconvolve(stim, np.ones((n_block, 1)), axes=0)[:n_times]
    dm_events = pd.DataFrame({
        'trial_type': [conditions[ii] for ii in idx[1]],
        'onset': idx[0] / raw_h.info['sfreq'],
        'duration': n_block / raw_h.info['sfreq']})
    dm = make_first_level_design_matrix(
        raw_h.times, dm_events, hrf_model='glover',
        drift_model='polynomial', drift_order=0)
    return stim, dm, events

#mne.viz.plot_events(events)

###############################################################################
# Select the example subject to plot

root1 = f'Day{day}'
root2 = f'{subject}_{day}'
root3 = f'*-*-*_*{run}'
fname_base = op.join(raw_path, root1, root2, root3)
print(fname_base)
fname = glob.glob(fname_base)
print(fname)
base = f'{subject}_{day}_00{run}'

raw_h_interp = mne.io.read_raw_fif(op.join(proc_path, f'{base}_hbo_raw.fif'))
use = None
assert use is None
use = dict(h=raw_h_interp,
        run=run,
        day=day)

assert isinstance(use, dict)
ch_names = [ch_name.rstrip(' hbo') for ch_name in use['h'].ch_names[::2]]
info = use['h'].info

# Plot the design matrix and some raw traces

fig, axes = plt.subplots(2, 1, figsize=(6., 3), constrained_layout=True)
# Design
ax = axes[0]
raw_h = use['h']
stim, dm, _ = _make_design(raw_h, design)

for ci, condition in enumerate(conditions):
    color = condition_colors[condition]
    ax.fill_between(
        raw_h.times, stim[:, ci], 0, edgecolor='none', facecolor='k',
        alpha=0.5)
    model = dm[conditions[ci]].to_numpy()
    ax.plot(raw_h.times, model, ls='-', lw=1, color=color)
    x = raw_h.times[np.where(model > 0)[0][0]]
    ax.text(
        x + 10, 1.1, condition, color=color, fontweight='bold', ha='center')
ax.set(ylabel='Modeled\noxyHb', xlabel='', xlim=raw_h.times[[0, -1]])

# HbO/HbR
ax = axes[1]
picks = [pi for pi, ch_name in enumerate(raw_h.ch_names)
         if 'S1_D2' in ch_name]
assert len(picks) == 2
fnirs_colors = dict(hbo='r', hbr='b')
#ylim = np.array([-2, 2])
for pi, pick in enumerate(picks):
    color = fnirs_colors[raw_h.ch_names[pick][-3:]]
    data = raw_h.get_data(pick)[0] * 1e6
    val = np.ptp(data)
    assert val > 0.01
    ax.plot(raw_h.times, data, color=color, lw=1.)
ax.set(xlabel='Time (s)', ylabel='μM',
#       ylim=ylim, 
       xlim=raw_h.times[[0, -1]])
#del raw_h
for ax in axes:
    for key in ('top', 'right'):
        ax.spines[key].set_visible(False)
#for ext in ('png', 'svg'):
#    fig.savefig(
#        op.join(
#            results_path, f'figure_1_{exp_name}.{ext}'))


data/Day3/215_3/*-*-*_*2
['data/Day3/215_3/2023-12-13_002']
Opening raw data file processed/215_3_002_hbo_raw.fif...
    Range : 0 ... 3975 =      0.000 ...   826.800 secs
Ready.
Used Annotations descriptions: ['1.0', '2.0', '3.0', '4.0', '5.0']


In [9]:
###############################################################################
# Run GLM analysis and epoching

sfreq = 4.807692050933838

df_cha = pd.DataFrame()
for day in days:
    for subject in subjects:
        # don't load existing files for now
        fname = op.join(proc_path, f'{subject}_{day}_{exp_name}.h5')
#        if not op.isfile(fname):
            # shift the following back over
        group = subject_to_group.get(int(subject), "unknown")
        subj_cha = pd.DataFrame()
        t0 = time.time()
        for run in runs:
            print(f'Running GLM for {group} {subject} day {day} run {run:03d}... ', end='')
            fname2 = op.join(proc_path, f'{subject}_{day}_{run:03d}_hbo_raw.fif')
            raw_h = mne.io.read_raw_fif(fname2)
            _, dm, _ = _make_design(raw_h, design, subject, run, day, group)
            glm_est = mne_nirs.statistics.run_glm(
                raw_h, dm, noise_model='ols', n_jobs=n_jobs)
            cha = glm_est.to_dataframe()
            cha['subject'] = subject
            cha['run'] = run
            cha['day'] = day
            cha['group'] = group
            cha['good'] = ~np.in1d(cha['ch_name'], bads)
            subj_cha = pd.concat([subj_cha, cha], ignore_index=True)
            del raw_h
        subj_cha.to_hdf(fname, 'subj_cha', mode='w')
        print(f'{time.time() - t0:0.1f} sec')
        # don't shift this
        df_cha = pd.concat([df_cha, pd.read_hdf(fname)], ignore_index=True)
df_cha.reset_index(drop=True, inplace=True)

# block averages
event_id = {condition: ci for ci, condition in enumerate(conditions, 1)}
evokeds = {condition: dict() for condition in conditions}
for day in days:
    for subject in subjects:
        fname = op.join(proc_path, f'{subject}_{day}_{exp_name}-ave.fif')
#        if not op.isfile(fname):
        # shift this forward:
        tmin, tmax = -2, 38
        baseline = (None, 0)
        t0 = time.time()
        print(f'Creating block average for {subject} day {day}... ', end='')
        raws = list()
        events = list()
        for run in runs:
            fname2 = op.join(proc_path, f'{subject}_{day}_{run:03d}_hbo_raw.fif')
            raw_h = mne.io.read_raw_fif(fname2)
            events.append(_make_design(raw_h, 'block', subject, run)[2])
            raws.append(raw_h)
        bads = sorted(set(sum((r.info['bads'] for r in raws), [])))
        for r in raws:
            r.info['bads'] = bads
        raw_h, events = mne.concatenate_raws(raws, events_list=events)
        epochs = mne.Epochs(raw_h, events, event_id, tmin=tmin, tmax=tmax,
                            baseline=baseline)
        this_ev = [epochs[condition].average() for condition in conditions]
        assert all(ev.nave > 0 for ev in this_ev)
        mne.write_evokeds(fname, this_ev, overwrite=True)
        print(f'{time.time() - t0:0.1f} sec')
        # don't shift this forward:
        for condition in conditions:
            evokeds[condition][subject] = mne.read_evokeds(fname, condition)

# Exclude bad channels
bad = dict()
bb = dict() # TEST

#for day in days:
#    for subject in subjects:
#        for run in runs:
#            fname2 = op.join(proc_path, f'{subject}_{day}_{run:03d}_hbo_raw.fif')
#            this_info = mne.io.read_info(fname2)
#            bad_channels = [idx - 1 for idx in sorted(
#                this_info['ch_names'].index(bad) + 1 for bad in this_info['bads'])]
#            valid_indices = np.arange(len(use['h'].ch_names))
#            bb = [b for b in bad_channels if b in valid_indices]
#            bad[(subject, run, day)] = bb
#        assert np.in1d(bad[(subject, run, day)], np.arange(len(use['h'].ch_names))).all()  # noqa: E501

# make life easier by combining across runs
bad_combo = dict()
for day in days:
    for (subject, run, day), bb in bad.items():
        bad_combo[subject] = sorted(set(bad_combo.get(subject, [])) | set(bb))
bad = bad_combo
#assert set(bad) == set(subjects)
start = len(df_cha)
n_drop = 0
for day in days:
    for (subject, run, day), bb in bad.items():
        if not len(bb):
            continue
        drop_names = [use['h'].ch_names[b] for b in bb]
        is_subject = (df_cha['subject'] == subject)
        is_day = (df_cha['day'] == day)
        assert len(is_subject) == len(df_cha)
        is_day = (df_cha['day'] == day)
        drop = df_cha.index[
            is_subject &
            is_day &
            np.in1d(df_cha['ch_name'], drop_names)]
        n_drop += len(drop)
        if len(drop):
            print(f'Dropping {len(drop)} for {subject} day {day}')  # {run}')
            df_cha.drop(drop, inplace=True)
end = len(df_cha)
assert n_drop == start - end, (n_drop, start - end)

# combine runs by averaging estimates
sorts = ['subject', 'ch_name', 'Chroma', 'Condition', 'group', 'day', 'run']
df_cha.sort_values(
    sorts, inplace=True)
assert (np.array(df_cha['run']).reshape(-1, 2) == runs).all()
theta = np.array(df_cha['theta']).reshape(-1, len(runs)).mean(-1)
df_cha.drop(
    [col for col in df_cha.columns if col not in sorts[:-1]], axis='columns',
    inplace=True)
df_cha.reset_index(drop=True, inplace=True)
df_cha = df_cha[::len(runs)]
df_cha.reset_index(drop=True, inplace=True)
df_cha['theta'] = theta
df_cha.to_csv(op.join(results_path, 'df_cha.csv'), index=False)

print("done :)")


Running GLM for trained 201 day 1 run 001... Opening raw data file processed/201_1_001_hbo_raw.fif...
    Range : 0 ... 3994 =      0.000 ...   830.752 secs
Ready.
Used Annotations descriptions: ['1.0', '2.0', '3.0', '4.0', '5.0']
Running GLM for trained 201 day 1 run 002... Opening raw data file processed/201_1_002_hbo_raw.fif...
    Range : 0 ... 3838 =      0.000 ...   798.304 secs
Ready.
Used Annotations descriptions: ['1.0', '2.0', '3.0', '4.0', '5.0']
6.1 sec
Running GLM for control 205 day 1 run 001... Opening raw data file processed/205_1_001_hbo_raw.fif...
    Range : 0 ... 4171 =      0.000 ...   867.568 secs
Ready.
Used Annotations descriptions: ['1.0', '2.0', '3.0', '4.0', '5.0']


/var/folders/4k/pq2dd_x1047dwxhfm5_xl8rw0000gn/T/ipykernel_38591/2837592040.py:32: FutureWarning: Starting with pandas version 3.0 all arguments of to_hdf except for the argument 'path_or_buf' will be keyword-only.
  subj_cha.to_hdf(fname, 'subj_cha', mode='w')


Running GLM for control 205 day 1 run 002... Opening raw data file processed/205_1_002_hbo_raw.fif...
    Range : 0 ... 4933 =      0.000 ...  1026.064 secs
Ready.
Used Annotations descriptions: ['1.0', '2.0', '3.0', '4.0', '5.0']
6.8 sec
Running GLM for trained 207 day 1 run 001... Opening raw data file processed/207_1_001_hbo_raw.fif...
    Range : 0 ... 3929 =      0.000 ...   817.232 secs
Ready.
Used Annotations descriptions: ['1.0', '2.0', '3.0', '4.0', '5.0']


/var/folders/4k/pq2dd_x1047dwxhfm5_xl8rw0000gn/T/ipykernel_38591/2837592040.py:32: FutureWarning: Starting with pandas version 3.0 all arguments of to_hdf except for the argument 'path_or_buf' will be keyword-only.
  subj_cha.to_hdf(fname, 'subj_cha', mode='w')


Running GLM for trained 207 day 1 run 002... Opening raw data file processed/207_1_002_hbo_raw.fif...
    Range : 0 ... 3958 =      0.000 ...   823.264 secs
Ready.
Used Annotations descriptions: ['1.0', '2.0', '3.0', '4.0', '5.0']
6.0 sec
Running GLM for control 208 day 1 run 001... Opening raw data file processed/208_1_001_hbo_raw.fif...
    Range : 0 ... 3941 =      0.000 ...   819.728 secs
Ready.
Used Annotations descriptions: ['1.0', '2.0', '3.0', '4.0', '5.0']


...

Running GLM for trained 219 day 3 run 002... Opening raw data file processed/219_3_002_hbo_raw.fif...
    Range : 0 ... 3924 =      0.000 ...   816.192 secs
Ready.
Used Annotations descriptions: ['1.0', '2.0', '3.0', '4.0', '5.0']
5.7 sec
Running GLM for control 224 day 3 run 001... Opening raw data file processed/224_3_001_hbo_raw.fif...
    Range : 0 ... 3952 =      0.000 ...   822.016 secs
Ready.
Used Annotations descriptions: ['1.0', '2.0', '3.0', '4.0', '5.0']


/var/folders/4k/pq2dd_x1047dwxhfm5_xl8rw0000gn/T/ipykernel_38591/2837592040.py:32: FutureWarning: Starting with pandas version 3.0 all arguments of to_hdf except for the argument 'path_or_buf' will be keyword-only.
  subj_cha.to_hdf(fname, 'subj_cha', mode='w')


Running GLM for control 224 day 3 run 002... Opening raw data file processed/224_3_002_hbo_raw.fif...
    Range : 0 ... 3945 =      0.000 ...   820.560 secs
Ready.
Used Annotations descriptions: ['1.0', '2.0', '3.0', '4.0', '5.0']
5.7 sec
Creating block average for 201 day 1... Opening raw data file processed/201_1_001_hbo_raw.fif...
    Range : 0 ... 3994 =      0.000 ...   830.752 secs
Ready.
Used Annotations descriptions: ['1.0', '2.0', '3.0', '4.0', '5.0']


/var/folders/4k/pq2dd_x1047dwxhfm5_xl8rw0000gn/T/ipykernel_38591/2837592040.py:32: FutureWarning: Starting with pandas version 3.0 all arguments of to_hdf except for the argument 'path_or_buf' will be keyword-only.
  subj_cha.to_hdf(fname, 'subj_cha', mode='w')


Opening raw data file processed/201_1_002_hbo_raw.fif...
    Range : 0 ... 3838 =      0.000 ...   798.304 secs
Ready.
Used Annotations descriptions: ['1.0', '2.0', '3.0', '4.0', '5.0']
Not setting metadata
40 matching events found
Setting baseline interval to [-2.0800001110839905, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Overwriting existing file.
2.2 sec
Reading /Users/wallacelab/Documents/Ansley/fnirs-analysis/processed/201_1_av-ave.fif ...
    Found the data of interest:
        t =   -2080.00 ...   38064.00 ms (A)
        0 CTF compensation matrices available
        nave = 10 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-2.08, 0] s)
Reading /Users/wallacelab/Documents/Ansley/fnirs-analysis/processed/201_1_av-ave.fif ...
    Found the data of interest:
        t =   -2080.00 ...   38064.00 ms (V)
        0 CTF compensation matrices ava

In [11]:
# Mixed linear model
def _mixed_df(ch_summary):
    formula = "theta ~ -1 + ch_name:Condition" 
    ch_model = smf.mixedlm(
        formula, ch_summary, groups=ch_summary["subject"]).fit(method='powell')
    ch_model_df = mne_nirs.statistics.statsmodels_to_results(ch_model)
    ch_model_df['P>|z|'] = ch_model.pvalues
    ch_model_df.drop([idx for idx in ch_model_df.index if '[constant]' in idx],
                    inplace=True)
    return ch_model_df

# Make separate subject lists for trained and untrained (TEST)
#trained_subjects = {'201 203 207 212 213 214 216 219 221 223'}
#control_subjects = {'202 204 205 206 208 209 215 217 218 224'}

# Run group level model and convert to dataframe
use_subjects = [subj for subj in subjects]
ch_summary = df_cha.query("Chroma in ['hbo']").copy()
#ch_summary = df_cha.query("Chroma in ['hbo'] and group in ['trained'] and day in ['3']").copy()
ch_summary_use = ch_summary.query(
    f"subject in {use_subjects}").copy()
ch_model_df = _mixed_df(ch_summary_use) 
ch_model_df.reset_index(inplace=True)

# Correct for multiple comparisons
print(f'Correcting for {len(ch_model_df["P>|z|"])} comparisons using FDR')
_, ch_model_df['P_fdr'] = mne.stats.fdr_correction(
    ch_model_df['P>|z|'], method='indep')
ch_model_df['SIG'] = ch_model_df['P_fdr'] < 0.05
ch_model_df.to_csv(op.join(results_path, 'ch_model_corrected_hbo.csv'), index=False)
ch_model_df.loc[ch_model_df.SIG == True]


/Users/wallacelab/Documents/Ansley/fnirs-analysis/.conda/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


Correcting for 336 comparisons using FDR


,index,Coef.,Std.Err.,z,P>|z|,[0.025,0.975],ch_name,Condition,Significant,P_fdr,SIG
3,ch_name[S11_D19 hbo]:Condition[A],-1.087566e-07,0.0,-4.852776,0.000001,-0.0,-0.0,S11_D19 hbo,A,True,0.000058,True
4,ch_name[S11_D21 hbo]:Condition[A],-6.677545e-08,0.0,-2.979555,0.002887,-0.0,-0.0,S11_D21 hbo,A,True,0.022681,True
9,ch_name[S12_D24 hbo]:Condition[A],-7.125870e-08,0.0,-3.179600,0.001475,-0.0,-0.0,S12_D24 hbo,A,True,0.015485,True
10,ch_name[S13_D21 hbo]:Condition[A],-7.828168e-08,0.0,-3.492969,0.000478,-0.0,-0.0,S13_D21 hbo,A,True,0.006173,True
11,ch_name[S13_D23 hbo]:Condition[A],-6.793478e-08,0.0,-3.031285,0.002435,-0.0,-0.0,S13_D23 hbo,A,True,0.020980,True
...,...,...,...,...,...,...,...,...,...,...,...,...
275,ch_name[S16_D27 hbo]:Condition[W],6.168290e-08,0.0,2.752323,0.005917,0.0,0.0,S16_D27 hbo,W,True,0.034385,True
279,ch_name[S18_D26 hbo]:Condition[W],5.824904e-08,0.0,2.599102,0.009347,0.0,0.0,S18_D26 hbo,W,True,0.049850,True
306,ch_name[S25_D13 hbo]:Condition[W],6.166044e-08,0.0,2.751321,0.005936,0.0,0.0,S25_D13 hbo,W,True,0.034385,True
324,ch_name[S6_D18 hbo]:Condition[W],6.360501e-08,0.0,2.838089,0.004538,0.0,0.0,S6_D18 hbo,W,True,0.031769,True


In [12]:
# Analyze HbR Data Separately

ch_summary_hbr = df_cha.query("Chroma in ['hbr']").copy()
ch_summary_use_hbr = ch_summary_hbr.query(
    f"subject in {use_subjects}").copy()
ch_model_df_hbr = _mixed_df(ch_summary_use_hbr) 
ch_model_df_hbr.reset_index(inplace=True)

# Correct for multiple comparisons
print(f'Correcting for {len(ch_model_df_hbr["P>|z|"])} comparisons using FDR')
_, ch_model_df_hbr['P_fdr'] = mne.stats.fdr_correction(
    ch_model_df_hbr['P>|z|'], method='indep')
ch_model_df_hbr['SIG'] = ch_model_df_hbr['P_fdr'] < 0.05
ch_model_df_hbr.to_csv(op.join(results_path, 'ch_model_corrected_hbr.csv'), index=False)
ch_model_df_hbr.loc[ch_model_df_hbr.SIG == True]

/Users/wallacelab/Documents/Ansley/fnirs-analysis/.conda/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


Correcting for 336 comparisons using FDR


,index,Coef.,Std.Err.,z,P>|z|,[0.025,0.975],ch_name,Condition,Significant,P_fdr,SIG
1,ch_name[S10_D20 hbr]:Condition[A],4.127714e-08,0.0,3.488120,4.864290e-04,0.0,0.0,S10_D20 hbr,A,True,0.013620,True
2,ch_name[S10_D22 hbr]:Condition[A],3.909874e-08,0.0,3.304035,9.530401e-04,0.0,0.0,S10_D22 hbr,A,True,0.020014,True
31,ch_name[S19_D8 hbr]:Condition[A],-4.576874e-08,0.0,-3.867683,1.098744e-04,-0.0,-0.0,S19_D8 hbr,A,True,0.006771,True
61,ch_name[S3_D2 hbr]:Condition[A],-3.479327e-08,0.0,-2.940202,3.279984e-03,-0.0,-0.0,S3_D2 hbr,A,True,0.048535,True
100,ch_name[S14_D26 hbr]:Condition[AV],-4.162366e-08,0.0,-3.517404,4.357905e-04,-0.0,-0.0,S14_D26 hbr,AV,True,0.013311,True
105,ch_name[S16_D25 hbr]:Condition[AV],-4.794279e-08,0.0,-4.051401,5.091191e-05,-0.0,-0.0,S16_D25 hbr,AV,True,0.005702,True
106,ch_name[S16_D26 hbr]:Condition[AV],-4.168136e-08,0.0,-3.522279,4.278531e-04,-0.0,-0.0,S16_D26 hbr,AV,True,0.013311,True
107,ch_name[S16_D27 hbr]:Condition[AV],-4.549161e-08,0.0,-3.844264,1.209146e-04,-0.0,-0.0,S16_D27 hbr,AV,True,0.006771,True
111,ch_name[S18_D26 hbr]:Condition[AV],-3.657358e-08,0.0,-3.090647,1.997210e-03,-0.0,-0.0,S18_D26 hbr,AV,True,0.035319,True
115,ch_name[S19_D8 hbr]:Condition[AV],-4.585452e-08,0.0,-3.874932,1.066546e-04,-0.0,-0.0,S19_D8 hbr,AV,True,0.006771,True


In [13]:
###############################################################################
# Plot significant channels
sig_chs = dict()
zs = dict()
for condition in conditions:
    sig_df = ch_model_df[
        (ch_model_df['P_fdr'] < 0.05) &
        (ch_model_df['Condition'] == condition) &
        (ch_model_df['ch_name'].isin(use['h'].ch_names))
        ]
    sig_chs[(condition)] = sorted(
        (use['h'].ch_names.index(row[1]['ch_name']), row[1]['P_fdr'])
        for row in sig_df.iterrows())
    ch_model_df[ch_model_df['Condition'] == condition]
    zs[condition] = np.array([
        ch_model_df.loc[(ch_model_df['Condition'] == condition) & 
                        (ch_model_df['ch_name'] == ch_name), 'z'].iloc[0]
        for ch_name in info['ch_names'][::2]], float)
    assert zs[condition].shape == (84,)
    assert np.isfinite(zs[condition]).all()

def _plot_sig_chs(sigs, ax):
    if sigs and isinstance(sigs[0], tuple):
        sigs = [s[0] for s in sigs]
    ch_groups = [sigs, np.setdiff1d(np.arange(info['nchan']), sigs)]
    mne.viz.plot_sensors(
        info, 'topomap', 'hbo', title='', axes=ax,
        show_names=True, ch_groups=ch_groups)
    ax.collections[0].set(lw=0)
    c = ax.collections[0].get_facecolor()
    c[(c[:, :3] == (0.5, 0, 0)).all(-1)] = (0., 0., 0., 0.1)
    c[(c[:, :3] == (0, 0, 0.5)).all(-1)] = (0., 1., 0., 0.5)
    ax.collections[0].set_facecolor(c)
    ch_names = [info['ch_names'][idx] for idx in sigs]
    texts = list(ax.texts)
    got = []
    for text in list(texts):
        try:
            idx = ch_names.index(text.get_text())
        except ValueError:
            text.remove()
        else:
            got.append(idx)
            text.set_text(f'{sigs[idx] // 2 + 1}')
            text.set(fontsize='xx-small', zorder=5, ha='center')
    assert len(got) == len(sigs), (got, list(sigs))

def _plot_sigs(sig_chs, all_corrs=()):
    n_col = max(len(x) for x in sig_chs.values()) + 1
    n_row = len(conditions)
    figsize = (n_col * 1.0, n_row * 1.0)
    fig, axes = plt.subplots(
        n_row, n_col, figsize=figsize, constrained_layout=True, squeeze=False)
    h_colors = {0: 'r', 1: 'b'}
    xticks = [0, 10, 20, 30]
    ylim = [-0.2, 0.3]
    yticks = [-0.2, -0.1, 0, 0.1, 0.2, 0.3]
    xlim = [times[0], 35]
    ylim = np.array(ylim)
    yticks = np.array(yticks)
    for ci, condition in enumerate(conditions):
        ii = 0
        sigs = sig_chs[condition]
        if len(sigs) == 0:
            sigs = [(None, None)]
        for ii, (ch_idx, ch_p) in enumerate(sigs):
            ax = axes[ci, ii]
            if ch_idx is not None:
                for jj in range(2):  # HbO, HbR
                    color = h_colors[jj]
                    a = 1e6 * np.array(
                        [evokeds[condition][subject].data[ch_idx + jj]
                         for subject in use_subjects
                         if ch_idx + jj not in bad.get(subject, [])], float)
                    m = np.mean(a, axis=0)
                    lower, upper = stats.t.interval(
                        0.95, len(a) - 1, loc=m, scale=stats.sem(a, axis=0))
                    ax.fill_between(
                        times, lower, upper, facecolor=color,
                        edgecolor='none', lw=0, alpha=0.25, zorder=3,
                        clip_on=False)
                    ax.plot(times, m, color=color, lw=1, zorder=4,
                            clip_on=False)
                # Correlations
                this_df = ch_summary_use.query(
                    f'ch_name == {repr(use["h"].ch_names[ch_idx])} and '
                    f'Chroma == "hbo" and '
                    f'Condition == {repr(condition)}')
                #assert 8 <= len(this_df) <= len(subjects), len(this_df)
                a = np.array(this_df['theta'])
                cs = list()
                if len(cs):
                    cs = [''] + cs
                c = '\n'.join(cs)
                ax.text(times[-1], ylim[1],
                        f'ch{ch_idx // 2 + 1}\np={ch_p:0.5f}{c}',
                        ha='right', va='top', fontsize='x-small')
            ax.axvline(20, ls=':', color='0.5', zorder=2, lw=1)
            ax.axhline(0, ls='-', color='k', zorder=2, lw=0.5)
            ax.set(xticks=xticks, yticks=yticks)
            ax.set(xlim=xlim, ylim=ylim)
            for key in ('top', 'right'):
                ax.spines[key].set_visible(False)
            if ax.get_subplotspec().is_last_row():
                ax.set(xlabel='Time (sec)')
            else:
                ax.set_xticklabels([''] * len(xticks))
            if ax.get_subplotspec().is_first_col():
                ax.set_ylabel(condition)
            else:
                ax.set_yticklabels([''] * len(yticks))
            for key in ('top', 'right'):
                ax.spines[key].set_visible(False)
        for ii in range(ii + 1, n_col - 1):
            fig.delaxes(axes[ci, ii])
        # montage
        ax = axes[ci, -1]
        if sigs[0][0] is None:
            fig.delaxes(ax)
        else:
            # plot montage
            _plot_sig_chs(sigs, ax)
    return fig

times = evokeds[conditions[0]][subjects[0]].times
info = evokeds[conditions[0]][subjects[0]].info
fig = _plot_sigs(sig_chs)
for ext in ('png', 'svg'):
    fig.savefig(op.join(results_path, f'stats_{exp_name}.{ext}'))


In [15]:
# ONLY PLOT SENSOR LOCATIONS
def _plot_sigs(sig_chs, all_corrs=()):
    n_col = 1  # Only need one column for sensor locations
    n_row = len(conditions)
    figsize = (n_col * 2, n_row * 2)  # Increase figure size for better label visibility
    fig, axes = plt.subplots(
        n_row, n_col, figsize=figsize, constrained_layout=True)

    # Handle the case of a single subplot
    if n_row == 1:
        axes = [axes]

    for ci, condition in enumerate(conditions):
        sigs = sig_chs[condition]
        ax = axes[ci]  # Direct reference to each subplot's axis
        
        # Ensure labels are set even if there are no significant sensors
        ax.set_ylabel(condition, labelpad=100)  # Increase labelpad if necessary

        # Only attempt to plot sensor locations if there are significant sensors
        if sigs[0][0] is not None:
            _plot_sig_chs(sigs, ax)
        else:
            # Optionally, clear the axes but keep them to display the label
            ax.clear()  # Clear the axes of any plotted data
            ax.set_xticks([])  # Remove x-ticks
            ax.set_yticks([])  # Remove y-ticks
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
            ax.spines['bottom'].set_visible(False)
            ax.spines['left'].set_visible(False)
    return fig

times = evokeds[conditions[0]][subjects[0]].times
info = evokeds[conditions[0]][subjects[0]].info
fig = _plot_sigs(sig_chs)


In [61]:
###############################################################################
# Source space projection

import pyvista
import pyvistaqt

info = use['h'].copy().pick_types(fnirs='hbo', exclude=()).info
info['bads'] = []
assert tuple(zs) == conditions

evoked = mne.EvokedArray(np.array(list(zs.values())).T, info)
picks = np.arange(len(evoked.ch_names))

for ch in evoked.info['chs']:
    assert ch['coord_frame'] == mne.io.constants.FIFF.FIFFV_COORD_HEAD
stc = mne.stc_near_sensors(
    evoked, trans='fsaverage', subject='fsaverage', mode='weighted',
    distance=0.02, project=True, picks=picks, subjects_dir=subjects_dir)
# Split channel indices by left lat, posterior, right lat:
num_map = {name: str(ii) for ii, name in enumerate(evoked.ch_names)}
evoked.copy().rename_channels(num_map).plot_sensors(show_names=True)
view_map = [np.arange(19), np.arange(19, 33), np.arange(33, 52)]
surf = mne.read_bem_surfaces(  # brain surface
    f'{subjects_dir}/fsaverage/bem/fsaverage-5120-5120-5120-bem.fif', s_id=1)

for ci, condition in enumerate(conditions):
    this_sig = [v[0] // 2 for v in sig_chs[condition]]
    #assert np.in1d(this_sig, np.arange(52)).all()
    pos = np.array([info['chs'][idx]['loc'][:3] for idx in this_sig])
    pos.shape = (-1, 3)  # can be empty
    trans = mne.transforms._get_trans('fsaverage', 'head', 'mri')[0]
    pos = mne.transforms.apply_trans(trans, pos)  # now in MRI coords
    pos = mne.surface._project_onto_surface(pos, surf, project_rrs=True)[2]
    # plot
    brain = stc.plot(hemi='both', views=['lat', 'frontal', 'lat'],
                    initial_time=evoked.times[ci], cortex='low_contrast',
                    time_viewer=False, show_traces=False,
                    surface='pial', smoothing_steps=0, size=(1200, 400),
                    clim=dict(kind='value', pos_lims=[0., 1.25, 2.5]),
                    colormap='RdBu_r', view_layout='horizontal',
                    colorbar=(0, 1), time_label='', background='w',
                    brain_kwargs=dict(units='m'),
                    add_data_kwargs=dict(colorbar_kwargs=dict(
                        title_font_size=12, label_font_size=12, n_labels=5,
                        title='z score')), subjects_dir=subjects_dir)
    brain.show_view(azimuth=270, elevation=90, row=0, col=1)
    brain.show_view('lat', hemi='rh', row=0, col=2)
    brain.show_view('lat', hemi='lh', row=0, col=0)
    pl = brain.plotter
#    plt.savefig(
#        op.join(results_path, f'all_brain_{exp_name}_{condition}.png'), pl.image)
    brain.save_image(
        op.join(results_path, f'all_brain_{exp_name}_{condition}.png'), brain)
    


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Projecting data from 84 sensors onto 327684 surface vertices: weighted mode
    Projecting sensors onto surface


    Minimum projected intra-sensor distance: 6.1 mm
    160208 / 327684 non-zero vertices
    3 BEM surfaces found


AttributeError: 'NoneType' object has no attribute 'screenshot'

In [17]:
# fOLD specificity
fold_files = ['10-10.xls', '10-5.xls']
for fname in fold_files:
    if not op.isfile(fname):
        pooch.retrieve(f'https://github.com/nirx/fOLD-public/raw/master/Supplementary/{fname}', None, fname, path=os.getcwd())  # noqa
raw_spec = use['h'].copy()
raw_spec.pick_channels(raw_spec.ch_names[::2])
specs = mne_nirs.io.fold_channel_specificity(raw_spec, fold_files, 'Brodmann')
for si, spec in enumerate(specs, 1):
    spec['Channel'] = si
    spec['negspec'] = -spec['Specificity']
specs = pd.concat(specs, ignore_index=True)
specs.drop(['Source', 'Detector', 'Distance (mm)', 'brainSens',
            'X (mm)', 'Y (mm)', 'Z (mm)'], axis=1, inplace=True)
specs.sort_values(['Channel', 'negspec'], inplace=True)
specs.drop('negspec', axis=1, inplace=True)
specs.reset_index(inplace=True, drop=True)
specs.to_csv(op.join(results_path, 'specificity.csv'), index=False)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


In [34]:
# Trying to plot data by condition... oof...

evokeds = dict(
            A = list(epochs['A'].iter_evoked()),
            V = list(epochs['V'].iter_evoked()),
            AV = list(epochs['AV'].iter_evoked()),
            control = list(epochs.group['control'].iter_evoked()),
            trained = list(epochs.group['trained'].iter_evoked()),
            )

# HbO_dict = {'A/HbO': epochs['A'].average(picks='hbo'),
#                'V/HbO': epochs['V'].average(picks='hbo'),
#                'AV/HbO': epochs['AV'].average(picks='hbo'),
#                 }

# for condition in HbO_dict:
#     HbO_dict[condition].rename_channels(lambda x: x[:-4])

color_dict = dict(A='red', V='blue', AV='green')
#group_dict = dict(control='dashed', trained='solid')

mne.viz.plot_compare_evokeds(evokeds, combine="mean", ci=0.95,
                            colors=color_dict, 
 #                           styles=group_dict,
                            )

#mne.viz.plot_compare_evokeds(HbR_dict, combine="mean", ci=0.95,
 #                           colors=color_dict, 
 #                           #styles=styles_dict
 #                          )


#evoked_A = epochs['A'].average()
#evoked_V = epochs['A_3'].average()
#evoked_diff = mne.combine_evoked([evoked_left, evoked_right], weights=[1, -1])
#evoked_diff.plot_topomap(ch_type='hbo', times=ts, axes=axes[0, 2:],
#                         vlim=vlim, colorbar=True, **topomap_args)



AttributeError: 'Epochs' object has no attribute 'group'